In [1]:

import tensorflow as tf
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import cv2 as cv
import funcs as ff
import os
from VAE import VAE
from datetime import datetime

# check that the environment is fine
print('TensorFlow version : ', tf.__version__ )
print('NumPy version : ', np.__version__ )
print('OpenCV version', cv.__version__)

print('gpu list : ', tf.config.list_physical_devices())

tf.random.set_seed(42)

TensorFlow version :  2.4.0-rc0
NumPy version :  1.21.0
OpenCV version 4.5.1
gpu list :  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
'''
DIRECTORY SETTINGS
'''
cwd = '/Volumes/Seagate Backup Plus Drive/EXPERIMENTS'

serie_dir = os.path.join(cwd, 'Embeddings')
if not os.path.isdir(serie_dir):
    os.makedirs(serie_dir)

exp_dir = os.path.join(serie_dir, 'TO-BE-DELETED')
if not os.path.isdir(exp_dir):
    os.makedirs(exp_dir)

log_dir = os.path.join(exp_dir, "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
if not os.path.isdir(log_dir):
    os.makedirs(log_dir)
time.sleep(2)
log_dir2 = os.path.join(exp_dir, "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
if not os.path.isdir(log_dir2):
    os.makedirs(log_dir2)

last_ckpt = os.path.join(exp_dir, 'last_ckpt')
if not os.path.isdir(last_ckpt):
  os.mkdir(last_ckpt)

print('cwd : ', cwd)
print('serie dir : ', serie_dir)
print('exp_dir : ', exp_dir)
print('logs dir : ', log_dir)
print('logs dir : ', log_dir2)
print('last ckpt dir : ', last_ckpt)

cwd :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS
serie dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings
exp_dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/TO-BE-DELETED
logs dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/TO-BE-DELETED/logs/fit/20210720-101904
logs dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/TO-BE-DELETED/logs/fit/20210720-101906
last ckpt dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/TO-BE-DELETED/last_ckpt


In [3]:
def record_parser(example):
    example_fmt = {
        'label': tf.io.FixedLenFeature([14], tf.float32),
        'image': tf.io.FixedLenFeature([],tf.string, default_value='')}
    parsed = tf.io.parse_single_example(example, example_fmt)
    image = tf.io.decode_png(parsed["image"],channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, parsed['label']

def normalize_image(img,labels):
    imagenet_mean = np.array([0.485, 0.456, 0.406])
    imagenet_std = np.array([0.229, 0.224, 0.225])
    img = (img - imagenet_mean) / imagenet_std
    return img,labels

def make_dataset(filename):
    base_path = '/Volumes/Seagate Backup Plus Drive/datasets/chexpert-tfrecords'
    full_path = os.path.join(base_path,filename)
    dataset = tf.data.TFRecordDataset(full_path)
    parsed_dataset = dataset.map(record_parser,num_parallel_calls = tf.data.experimental.AUTOTUNE)
    #parsed_dataset = parsed_dataset.map(normalize_image,num_parallel_calls = tf.data.experimental.AUTOTUNE)
    return parsed_dataset

batch_size = 5
train_ds = make_dataset('new_training_cropped.tfrecords').shuffle(buffer_size=batch_size*7).batch(batch_size, drop_remainder=True).prefetch(1)
val_ds = make_dataset('validation_cropped.tfrecords').shuffle(buffer_size=100).batch(batch_size, drop_remainder=True).prefetch(1)
test_ds = make_dataset('test_set_cropped.tfrecords').shuffle(buffer_size=50).batch(batch_size, drop_remainder=False).prefetch(1)

label_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
                'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [4]:
# ###############
# Model
# ###############
latent_dim = 200
vae = VAE(number_of_classes=0,
          latent_dim=latent_dim,
          input_shape=(224, 224, 3),
          backbone=tf.keras.applications.DenseNet121(include_top=False, pooling='avg'))
vae.encoder.load_weights('/Volumes/Seagate Backup Plus Drive/models/ChestXray/ConditionalTraining/DenseNet121/FineTuning/model-05.hdf5', by_name=True)
#vae.summary([224, 224, 3])
#tf.keras.utils.plot_model(
#    vae.vae, to_file=os.path.join(exp_dir, 'DenseNet121.png'), show_shapes=True, show_dtype=True,
#    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96
#)

In [5]:
'''
# ###############
# TRAINING LOOP FUNCTIONS
# ###############
'''
lr = 7.5e-4
kl_weight = 5e-4
epochs = 10
optimizer = tf.keras.optimizers.Adam(lr)
writer = tf.summary.create_file_writer(logdir=log_dir)
loss = tf.keras.metrics.Mean(name='Training-Loss', dtype=tf.float32)
latent_loss = tf.keras.metrics.Mean(name='Training-LatentLoss', dtype=tf.float32)
rec_loss = tf.keras.metrics.Mean(name='Training-RecontructionLoss', dtype=tf.float32)
val_loss = tf.keras.metrics.Mean(name='Validation-Loss', dtype=tf.float32)
val_rec_loss = tf.keras.metrics.Mean(name='Validation-RecontructionLoss', dtype=tf.float32)
val_latent_loss = tf.keras.metrics.Mean(name='Validation-LatentLoss', dtype=tf.float32)

# TRAINING STEP
@tf.function
def vae_train_step(x, optimizer, kl_weight = 5e-4):

  with tf.GradientTape() as tape:
    # Feed input x into dbvae. Note that this is using the DB_VAE call function!
    x_rec, z_mean, z_logsigma = vae(x)
    loss, rec_loss, latent_loss = VAE.loss(x, x_rec, z_mean, z_logsigma, kl_weight)
    # classification_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=y), axis=-1)
  grads = tape.gradient(loss, vae.trainable_variables)
  # apply gradients to variables
  optimizer.apply_gradients(zip(grads, vae.trainable_variables))
  return loss, rec_loss, latent_loss

# CALLBACKS
class ImgPlotCB(tf.keras.callbacks.Callback):
    """Custom callback to sistematically plot images"""
    def __init__(self,
                 samples = tf.zeros((128, 128)),
                 model = None,
                 plot_freq = 200
                 ):
        super(ImgPlotCB, self).__init__()
        if model is not None:
            self.model = model
        self.orig_imgs = samples
        self.orig_normlzd = ff.normalize_tensor(samples)
        self.plot_freq = plot_freq
        self.x_rec = None
        self.z_mean = None
        self.z_logsigma = None

    def on_train_begin(self, logs=None):
        return

    def on_train_end(self, logs=None):
        return

    def on_batch_begin(self, batch, logs=None):
        return

    def on_batch_end(self, batch, logs=None):
        if not bool(batch % self.plot_freq):
            self._plot_imgs()
        return

    def on_epoch_begin(self, epoch, logs=None):
        return

    def on_epoch_end(self, epoch, logs=None):
        display.clear_output(wait=False)
        # sanity check for mu and sigma
        print('z_mean mean : {}'.format(tf.math.reduce_mean(self.z_mean)))
        print('z_logsigma mean : {}'.format(tf.math.reduce_mean(self.z_logsigma)))
        stddev = tf.math.exp(self.z_logsigma)

        # sample three times to sanity-check the decoding
        for i in range(3):
            z_new = tf.random.normal(shape= tf.shape(self.z_mean), mean=self.z_mean, stddev=stddev)
            new_rec = self.model.decode(z_new)
            new_rec = ff.normalize_tensor(new_rec)
            fig = plt.figure(i+3)
            plt.title('New Sampling {}'.format(i), color='red')
            for i in range(new_rec.shape[0]):
                plt.subplot(1, 4, i + 1)
                plt.imshow(new_rec[i, :, :, :])
                plt.axis('off')
        self._plot_imgs()

        return

    def _plot_imgs(self):
        self.x_rec, self.z_mean, self.z_logsigma = self.model(self.orig_imgs)
        img = ff.normalize_tensor(self.x_rec)

        fig = plt.figure(1)
        plt.title('Images', color='red')
        for i in range(img.shape[0]):
            plt.subplot(1, 4, i + 1)
            plt.imshow(img[i, :, :, :])
            plt.axis('off')

        fig = plt.figure(2)
        plt.title('Original Images', color='green')
        for i in range(self.orig_normlzd.shape[0]):
            plt.subplot(1, 4, i + 1)
            plt.imshow(self.orig_normlzd[i, :, :, :])
            plt.axis('off')
        plt.show()

    # ###################
    # CUSTOM CALLBACK END
    # ###################

class KLWeightScheduler(tf.keras.callbacks.Callback):
    """Custom callback to increase the kl regularization effect, making it sleep
    for a few epochs at the start"""
    def __init__(self,
                 initial_weight = 1e-3,
                 patience = 10,
                 max_value = 0.5,
                 factor = 1.1
                 ):
        super(KLWeightScheduler, self).__init__()
        self.patience = patience
        self.KL_weight = 0.
        self.max_val = max_value
        self.factor = factor
        self.initial_weight = initial_weight

    def on_train_begin(self, logs=None):
        return

    def on_train_end(self, logs=None):
        return

    def on_batch_begin(self, batch, logs=None):
        return

    def on_batch_end(self, batch, logs=None):
        return

    def on_epoch_begin(self, epoch, logs=None):
        if epoch == (self.patience):
            self.KL_weight = self.initial_weight
        elif epoch > (self.patience):
            self.KL_weight = self.initial_weight * (self.factor**epoch)
            self.KL_weight = tf.math.minimum(self.KL_weight, self.max_val)
        logs['KL_weight'] = self.KL_weight
        print('Kullback-Leibler weight : {}'.format(self.KL_weight))
        return

    def on_epoch_end(self, epoch, logs=None):
        return

    # ###################
    # CUSTOM CALLBACK END
    # ###################

class ReduceLROnPlateau(tf.keras.callbacks.Callback):

    def __init__(self,
                  factor=0.1,
                  patience=10,
                  verbose=0,
                  mode='auto',
                  min_delta=1e-4,
                  cooldown=0,
                  min_lr=0,
                  sign_number = 4,
                  optim_lr = None,
                  reduce_lin = False):
        super(ReduceLROnPlateau, self).__init__()
        # Optimizer Error Handling
        if not tf.is_tensor(optim_lr):
            raise ValueError('Need optimizer !')
        if factor >= 1.0:
            raise ValueError('ReduceLROnPlateau ' 'does not support a factor >= 1.0.')
        # Passing optimizer as arguement
        self.optim_lr = optim_lr
        self.factor = factor
        self.min_lr = min_lr
        self.min_delta = min_delta
        self.patience = patience
        self.verbose = verbose
        self.cooldown = cooldown
        self.cooldown_counter = 0  # Cooldown counter.
        self.wait = 0
        self.best = 0
        self.mode = mode
        self.monitor_op = None
        self.sign_number = sign_number


        ## Custom modification: linearly reducing learning
        self.reduce_lin = reduce_lin
        self.reduce_lr = True

        self._reset()

    def _reset(self):
        """Resets wait counter and cooldown counter.
        """
        if self.mode not in ['auto', 'min', 'max']:
            print('Learning Rate Plateau Reducing mode %s is unknown, '
                            'fallback to auto mode.', self.mode)
            self.mode = 'auto'
        if self.mode == 'min' or (self.mode == 'auto'):
            self.monitor_op = lambda a, b: np.less(a, b - self.min_delta)
            self.best = np.Inf
        else:
            self.monitor_op = lambda a, b: np.greater(a, b + self.min_delta)
            self.best = -np.Inf
        self.cooldown_counter = 0
        self.wait = 0

    def in_cooldown(self):
        return self.cooldown_counter > 0

    def on_epoch_end(self, epoch, loss, logs=None):
        logs = logs or {}
        ## Custom modification: Optimizer
        # logs['lr'] = K.get_value(self.model.optimizer.lr) returns a numpy array
        # and therefore can be modified to
        logs['lr'] = float(self.optim_lr.numpy())

        ## Custom modification: Deprecated due to focusing on validation loss
        # current = logs.get(self.monitor)

        current = float(loss)

        ## Custom modification: Deprecated due to focusing on validation loss
        # if current is None:
        #     print('Reduce LR on plateau conditioned on metric `%s` '
        #                     'which is not available. Available metrics are: %s',
        #                     self.monitor, ','.join(list(logs.keys())))

        # else:

        if self.in_cooldown():
            self.cooldown_counter -= 1
            self.wait = 0

        if self.monitor_op(current, self.best):
            self.best = current
            self.wait = 0
        elif not self.in_cooldown():
            self.wait += 1
            if self.wait >= self.patience:

                ## Custom modification: Optimizer Learning Rate
                # old_lr = float(K.get_value(self.model.optimizer.lr))
                old_lr = float(self.optim_lr.numpy())
                if old_lr > self.min_lr and self.reduce_lr == True:
                    ## Custom modification: Linear learning Rate
                    if self.reduce_lin == True:
                        new_lr = old_lr - self.factor
                        ## Custom modification: Error Handling when learning rate is below zero
                        if new_lr <= 0:
                            print('Learning Rate is below zero: {}, '
                            'fallback to minimal learning rate: {}. '
                            'Stop reducing learning rate during training.'.format(new_lr, self.min_lr))
                            self.reduce_lr = False
                    else:
                        new_lr = old_lr * self.factor
                    new_lr = max(new_lr, self.min_lr)
                    print('Learning rate has been reduced.'
                        'Old learning rate : {}'
                        'New Learning rate : {}'.format(old_lr, new_lr))


                    ## Custom modification: Optimizer Learning Rate
                    # K.set_value(self.model.optimizer.lr, new_lr)
                    self.optim_lr.assign(new_lr)

                    if self.verbose > 0:
                        print('\nEpoch %05d: ReduceLROnPlateau reducing learning '
                                'rate to %s.' % (epoch + 1, float(new_lr)))
                    self.cooldown_counter = self.cooldown
                    self.wait = 0


class LRScheduler(tf.keras.callbacks.Callback):
    def __init__(self, learning_rate, schedule, max_val=None, min_val=None):
        super(LRScheduler, self).__init__()
        self.lr = learning_rate
        self.schedule = schedule
        self.max = max_val
        self.min = min_val

    def on_epoch_begin(self, epoch, logs=None):
        tmp = self.schedule(epoch, float(self.lr.numpy()))
        tmp = tf.math.maximum(tmp, self.min) if self.min is not None else tmp
        tmp = tf.math.minimum(tmp, self.max) if self.max is not None else tmp
        self.lr.assign(tmp)
        logs['lr'] = float(self.lr.numpy())
        print('Current Learnig Rate : {}'.format(logs['lr']))

class Logger(tf.keras.callbacks.Callback):
    def __init__(self,
                 logdir : str,
                 logdir2 : str,
                 scalars : list,
                 names : list,
                 batch_scalars : list = None,
                 batch_scal_names : list = None,
                 batch_log_freq = 300):
        super(Logger, self).__init__()
        self.scalars = scalars
        self.names = names
        self.batch_scal = batch_scalars
        self.batch_scal_names = batch_scal_names
        self.batch_log_freq = batch_log_freq
        self.batch_writer = tf.summary.create_file_writer(logdir)
        self.epoch_writer = tf.summary.create_file_writer(logdir2)

    def on_batch_end(self, batch, logs=None):
        if not bool(batch%self.batch_log_freq):
            print('Step : {}'.format(batch))
            for val, name in zip(self.scalars, self.names):
                print(name, val.result())
                with self.batch_writer.as_default():
                  tf.summary.scalar(data=val.result(), name=name, step=batch)


    def on_epoch_end(self, epoch, logs=None):
        print('keys : ', list(logs.keys()))
        for val, name in zip(self.scalars, self.names):
            if name == 'loss':
                logs['loss'] = float(val.result().numpy())
        '''with self.epoch_writer.as_default():
            tf.summary.scalar(data=val.result(), name=name, step=epoch)
    for key, value in logs:
        with self.epoch_writer.as_default():
            tf.summary.scalar(key, value, epoch)'''


class SaveModels(tf.keras.callbacks.Callback):
    def __init__(self, model : tf.keras.Model, name : str):
        super(SaveModels, self).__init__()
        self.model = model
        self.name = name

    def on_batch_end(self, batch, logs=None):
        if not bool((batch+1)%300):
            self.model.vae.save(os.path.join(last_ckpt, self.name+'-last'), save_format='h5')

    def on_epoch_end(self, epoch, logs=None):
        self.model.vae.save(os.path.join(exp_dir, self.name+'_epoch_{}'.format(epoch)), save_format='h5')

def lr_schedule(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr*tf.math.exp(-0.2)
lrscheduler_cb = LRScheduler(learning_rate=optimizer.learning_rate, schedule=lr_schedule)
tb_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, write_graph=False, write_images=False, update_freq=100)
tb_cb.set_model(vae)

rnd = tf.random.uniform(shape=(), minval=3, maxval=batch_size, dtype=tf.dtypes.int32)
tmp = train_ds.take(1)
examples = []
for i, ex in enumerate(tmp):
    examples.append(ex[0][rnd, ...])
    examples.append(ex[0][rnd-1, ...])
    examples.append(ex[0][rnd-2, ...])
    examples.append(ex[0][rnd-3, ...])
examples = tf.convert_to_tensor(examples)
imgplot_cb = ImgPlotCB(samples=examples, model=vae, plot_freq=300)
progbar_cb = tf.keras.callbacks.ProgbarLogger('steps')
klweight_cb = KLWeightScheduler(initial_weight=kl_weight, patience=3, factor=1.5)
reducelronplateau_cb = ReduceLROnPlateau(factor=0.5, patience=0, reduce_lin=False, optim_lr=optimizer.learning_rate, cooldown=1, min_delta=100)
savemodels_cb = SaveModels(vae, 'VAE{}-DenseNet121-'.format(latent_dim))
log_cb = Logger(log_dir,
                log_dir2,
                scalars=[latent_loss, rec_loss, loss, val_latent_loss, val_rec_loss, val_loss],
                names=['latent loss', 'reconstruction loss', 'loss', 'validation latent loss', 'validation reconstruction loss', 'validation loss'])
                # batch_scalars=[latent_loss, rec_loss, loss],
                # batch_scal_names=['latent loss-b', 'reconstruction loss-b', 'loss-b'])

cb_list = [log_cb]
callback_list = tf.keras.callbacks.CallbackList(callbacks=cb_list, model=vae)

2021-07-20 10:19:07.967127: I tensorflow/python/profiler/internal/profiler_wrapper.cc:182] Profiling will start immediately because delay_ms was unset or zero.
2021-07-20 10:19:07.967141: I tensorflow/core/profiler/lib/profiler_session.cc:133] Profiler session started.
2021-07-20 10:19:07.972034: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-20 10:19:07.972138: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


In [6]:
'''TRAINING LOOP'''

callback_list.on_train_begin()
fake_ds = train_ds.take(3)
reducelronplateau_cb.on_train_begin()
for epoch in tqdm(range(epochs)):
    callback_list.on_epoch_begin(epoch)
    start_time = time.time()
    for batch, (train_x, train_labels) in enumerate(fake_ds):
        callback_list.on_batch_begin(batch)
        l, rl, ll = vae_train_step(train_x, optimizer, klweight_cb.KL_weight)

        #metrics
        loss(l)
        rec_loss(rl)
        latent_loss(ll)

        callback_list.on_batch_end(batch)
    print('train step time : {}'.format(time.time() - start_time))
    reducelronplateau_cb.on_epoch_end(epoch, float(loss.result().numpy()))
    print('Learning Rate Check : {}'.format(float(optimizer.learning_rate.numpy())))
    start_time = time.time()
    for test_x, test_labels in val_ds.take(5):
        x_rec, z_mean, z_logsigma = vae(test_x)
        l, rl, ll = vae.loss(test_x, x_rec, z_mean, z_logsigma)

        #metrics
        val_loss(l)
        val_rec_loss(rl)
        val_latent_loss(ll)

    print('val step time : {}'.format(time.time() - start_time))

    callback_list.on_epoch_end(epoch)

    # reset metrics
    val_loss.reset_states()
    val_rec_loss.reset_states()
    val_latent_loss.reset_states()
    loss.reset_states()
    latent_loss.reset_states()
    rec_loss.reset_states()

callback_list.on_train_end()

  0%|          | 0/10 [00:00<?, ?it/s]

Step : 0
latent loss tf.Tensor(108.159195, shape=(), dtype=float32)
reconstruction loss tf.Tensor(76880.82, shape=(), dtype=float32)
loss tf.Tensor(76880.82, shape=(), dtype=float32)
validation latent loss tf.Tensor(0.0, shape=(), dtype=float32)
validation reconstruction loss tf.Tensor(0.0, shape=(), dtype=float32)
validation loss tf.Tensor(0.0, shape=(), dtype=float32)
train step time : 3.586151123046875
Learning Rate Check : 0.000750000006519258


 10%|█         | 1/10 [00:06<01:00,  6.77s/it]

val step time : 3.1798110008239746
keys :  []
Step : 0
latent loss tf.Tensor(104.12959, shape=(), dtype=float32)
reconstruction loss tf.Tensor(68467.625, shape=(), dtype=float32)
loss tf.Tensor(68467.625, shape=(), dtype=float32)
validation latent loss tf.Tensor(0.0, shape=(), dtype=float32)
validation reconstruction loss tf.Tensor(0.0, shape=(), dtype=float32)
validation loss tf.Tensor(0.0, shape=(), dtype=float32)
train step time : 1.2536101341247559
Learning Rate Check : 0.000750000006519258


 20%|██        | 2/10 [00:10<00:42,  5.25s/it]

...

val step time : 2.849493980407715
keys :  []
Step : 0
latent loss tf.Tensor(245.34053, shape=(), dtype=float32)
reconstruction loss tf.Tensor(28545.25, shape=(), dtype=float32)
loss tf.Tensor(28545.25, shape=(), dtype=float32)
validation latent loss tf.Tensor(0.0, shape=(), dtype=float32)
validation reconstruction loss tf.Tensor(0.0, shape=(), dtype=float32)
validation loss tf.Tensor(0.0, shape=(), dtype=float32)
train step time : 1.0854759216308594
Learning Rate Check : 0.0001875000016298145


 90%|█████████ | 9/10 [00:38<00:04,  4.04s/it]

val step time : 2.837679147720337
keys :  []
Step : 0
latent loss tf.Tensor(240.96245, shape=(), dtype=float32)
reconstruction loss tf.Tensor(32914.14, shape=(), dtype=float32)
loss tf.Tensor(32914.14, shape=(), dtype=float32)
validation latent loss tf.Tensor(0.0, shape=(), dtype=float32)
validation reconstruction loss tf.Tensor(0.0, shape=(), dtype=float32)
validation loss tf.Tensor(0.0, shape=(), dtype=float32)
train step time : 1.1079869270324707
Learning Rate Check : 0.0001875000016298145


100%|██████████| 10/10 [00:42<00:00,  4.28s/it]

val step time : 2.920198917388916
keys :  []
